In [1]:
import os
os.environ['OMP_NUM_THREADS'] = "1"
os.environ['MKL_NUM_THREADS'] = "1"
os.environ['OPENBLAS_NUM_THREADS'] = "1"

import pandas as pd
import numpy as np
from tsfresh import extract_features, select_features, extract_relevant_features
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.utilities.dataframe_functions import impute

import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore, boxcox
import math


/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 9.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [61]:
class Window:
    def __init__(self, targets, window_size=100):
        self.targets = targets
        self.window_size = window_size

        self.start = 0
        self.end = self.window_size


    def get(self):
        return self.targets[self.start: self.end]

    def next(self):
        self.start += 1
        self.end += 1

    def hasNext(self):
        if self.end < len(self.targets):
            return True

        return False

In [3]:
class Building:
    def __init__(self, train_x, train_y, val_x, val_y, test_x):
        self.train_x = train_x
        self.train_y = train_y
        
        self.val_x = val_x
        self.val_y = val_y
        
        self.test_x = test_x
        

In [60]:
class Data:
    
    def __init__(self, train_a, train_b, train_c, observed_a, 
                 observed_b, observed_c, estimated_a, estimated_b, estimated_c, test_a, test_b, test_c):
        
        self.observed_a = self.quartersToHours(observed_a)
        self.observed_b = self.quartersToHours(observed_b)
        self.observed_c = self.quartersToHours(observed_c)
        
        self.estimated_a = self.quartersToHours(estimated_a)
        self.estimated_a = self.estimated_a.drop(["date_calc"], axis=1)
        
        self.estimated_b = self.quartersToHours(estimated_b)
        self.estimated_b = self.estimated_b.drop(["date_calc"], axis=1)
        
        self.estimated_c = self.quartersToHours(estimated_c)
        self.estimated_c = self.estimated_c.drop(["date_calc"], axis=1)

        self.test_a = self.quartersToHours(test_a)
        self.test_a = self.test_a.drop(["date_calc"], axis=1)
        self.test_a = self.test_a.rename(columns={'date_forecast': 'date'})
        
        self.test_b = self.quartersToHours(test_b)
        self.test_b = self.test_b.drop(["date_calc"], axis=1)
        self.test_b = self.test_b.rename(columns={'date_forecast': 'date'})

        self.test_c = self.quartersToHours(test_c)
        self.test_c = self.test_c.drop(["date_calc"], axis=1)
        self.test_c = self.test_c.rename(columns={'date_forecast': 'date'})
        
        
        self.train_a = train_a
        self.train_b = train_b
        self.train_c = train_c
        
        self.data_a = self.join_data(self.observed_a, self.estimated_a, self.train_a)
        self.data_b = self.join_data(self.observed_b, self.estimated_b, self.train_b)
        self.data_c = self.join_data(self.observed_c, self.estimated_c, self.train_c)
        
                
        self.data_a = self.data_a.fillna(0)
        self.test_a = self.test_a.fillna(0)

        self.data_b = self.data_b.fillna(0)
        self.test_b = self.test_b.fillna(0)

        self.data_c = self.data_c.fillna(0)
        self.test_c = self.test_c.fillna(0)
        
        
        self.training_data = [self.data_a, self.data_b, self.data_c]
        self.test_data = [self.test_a, self.test_b, self.test_c]
        
        
        self.process()

    
    
    def join_data(self, observed, estimated, labels):
        #Remove hour and minute values
        observed = observed.assign(date_forecast=observed.date_forecast.dt.round('H'))
        estimated = estimated.assign(date_forecast=estimated.date_forecast.dt.round('H'))

        #rename columns names to match
        observed = observed.rename(columns={'date_forecast': 'date'})
        estimated = estimated.rename(columns={'date_forecast': 'date'})
        labels = labels.rename(columns={'time': 'date'})

        data = pd.concat([observed, estimated])
        joined_data = pd.merge(data, labels, how="inner", on="date")

        return joined_data


    def quartersToHours(self, df):
        df['date_forecast'] = pd.to_datetime(df['date_forecast'], format='%Y-%m.%d %H:%M:%S')
        df["year"] = df['date_forecast'].dt.year
        df["month"] = df['date_forecast'].dt.month
        df["day"] = df['date_forecast'].dt.day
        df["hour"] = df['date_forecast'].dt.hour


        group = df.groupby([df["year"], df["month"], df["day"], df["hour"]])  
        result = group.mean()
        result = result.reset_index()

        return_df = result.drop(['year','month', 'day', 'hour'], axis=1)

        return return_df
    
    def find_and_delete_constants(self, df):
        WINDOW_SIZE = 24

        window = Window(df["pv_measurement"], WINDOW_SIZE)

        constants = []

        while window.hasNext():
            if window.get().std() <= 0.1:
                constants.append((window.start, window.end))
            window.next()

        def extract_indices(ranges):
            return [start for start, _ in ranges]


        indices = extract_indices(constants)


        df = df.drop(indices)
        df = df.reset_index()
        df = df.drop(['index'], axis=1)

        return df
    
    def convert_date_to_sin_and_cos(self, df):
        day = 24*60*60 #seconds in a day
        year = (365.2425)*day #seconds in a year
        month = year / 12.0

        date_time = pd.to_datetime(df['date'], format='%Y-%m.%d %H:%M:%S')

        timestamp_s = date_time.map(pd.Timestamp.timestamp)

        #df['Month sin'] = np.sin(timestamp_s * (2 * np.pi / month))
        #df['Month cos'] = np.cos(timestamp_s * (2 * np.pi / month))

        df['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
        df['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

        df['Hour sin'] = np.sin(timestamp_s * (2 * np.pi / day))
        df['Hour cos'] = np.cos(timestamp_s * (2 * np.pi / day))
        
        return df
        
    def round_is_day(self, df):
        df["is_day:idx"] = [int(round(df["is_day:idx"][i])) for i in range(len(df))]
        df["is_in_shadow:idx"] = [int(round(df["is_in_shadow:idx"][i])) for i in range(len(df))]
        
        return df
    
    
    def combine_features(self, df):
        
        new_features = [
            [['direct_rad:W', 'diffuse_rad:W'], 'global_rad:W'],
            [['direct_rad_1h:J', 'diffuse_rad_1h:J'], 'global_rad_1h:J']]
        
        for features, new_name in new_features:
    
            totals = []

            for i in range(len(df)):
                total = 0
                for feature in features:
                    total += df[feature][i]

                totals.append(total)
            df[new_name] = totals
            
        return df
    
    def remove_unused_columns(self, df):
        df = df.drop(['snow_drift:idx', 'elevation:m', "rain_water:kgm2", "rain_water:kgm2",
                      "wind_speed_w_1000hPa:ms"], axis=1)
        return df
    
    def split_data(self, df):
        val_split = 0.2

        len_df = len(df)
        train_len = int(len_df * (1.0 - val_split))

        train = df.iloc[:train_len,:]
        val = df.iloc[train_len:,:]


        train = train.reset_index()
        train = train.drop(['index'], axis=1)

        val = val.reset_index()
        val = val.drop(['index'], axis=1)

        return train, val
    
    def createFeatures(self, df): 
        top5 = ['absolute_humidity_2m:gm3', 'ceiling_height_agl:m', 'air_density_2m:kgm3', 'cloud_base_agl:m', 'sun_azimuth:d'] 
        seenKombos = []
        toCombine = [('effective_cloud_cover:p', 'clear_sky_energy_1h:J')]
        for elem in top5:
            for elem_2 in top5:
                if (elem, elem_2) not in seenKombos:
                    seenKombos.append((elem, elem_2))
                    if elem != elem_2:
                        toCombine.append((elem, elem_2))

        for col in df:
            if col == "date":
                continue
            for col_2 in df:
                if col_2 == "date":
                    continue
                if (col, col_2) not in seenKombos:
                    seenKombos.append((col, col_2))
                    if col != col_2 and (col in top5 or col_2 in top5):
                        if abs(df[col].corr(df[col_2])) >= 0.8:
                            toCombine.append((col, col_2))

        #print("toComb: ", toCombine)
        #print("------------------------")
        created = []
        for i in range(len(toCombine)):
            df[toCombine[i][0] + '_' + toCombine[i][1]] = df[toCombine[i][0]] * df[toCombine[i][1]]
        return df
    
    def apply(self, data, function):
        for i in range(len(data)):
            data[i] = function(data[i])
            
    def process(self):
        #PV constants
        self.apply(self.training_data, self.find_and_delete_constants)
  
        
        #Time sin and cos
        self.apply(self.training_data, self.convert_date_to_sin_and_cos)
        self.apply(self.test_data, self.convert_date_to_sin_and_cos)
        
        #Round isday and isshadow
        self.apply(self.training_data, self.round_is_day)
        self.apply(self.test_data, self.round_is_day)
        
        #combine
        self.apply(self.training_data, self.combine_features)
        self.apply(self.test_data, self.combine_features)
        
        #fill Nan
        for i in range(len(self.training_data)):
            self.training_data[i] = self.training_data[i].fillna(0)
            
        for i in range(len(self.test_data)):
            self.test_data[i] = self.test_data[i].fillna(0)
        
        #remove unsused
        self.apply(self.training_data, self.remove_unused_columns)
        self.apply(self.test_data, self.remove_unused_columns)
    
        
        for i in range(len(self.training_data)):
            z = self.training_data[i].drop("date", axis=1).apply(zscore)
            
            hasBigOutliers = []
            for col in z:
                if col == "date" or col == "pv_measurement":
                    continue
                for val in z[col]:
                    if val >= 10:
                        hasBigOutliers.append(col)
                        break
                    
            for col in hasBigOutliers:
                self.training_data[i][col] = boxcox(self.training_data[i][col] + 1)[0]
                if self.test_data[i][col].std() != 0.0:
                    self.test_data[i][col] = boxcox(self.test_data[i][col] + 1)[0]  
  
        for i in range(len(self.training_data)):
            temp = self.training_data[i].pop("pv_measurement")
            temp_data = pd.concat([self.training_data[i], self.test_data[i]])
            res = self.createFeatures(temp_data)
            
            train = res[:len(temp)]
            train["pv_measurement"] = temp
            test = res[len(temp):]
            
            self.training_data[i] = train
            self.test_data[i] = test
            
            
        new_train = []
        new_test = []
        for i in range(len(self.training_data)):
            print(f'tsfresh feature extraction on building {["A", "B", "C"][i]}')
            train_with_index = self.training_data[i].reset_index()
            test_with_index = self.test_data[i].reset_index()

            extracted_train_features = extract_features(train_with_index[["date", "index", "global_rad:W"]], 
                                                        column_id="index", 
                                                        column_sort="date", 
                                                        default_fc_parameters=EfficientFCParameters())
            
            extracted_test_features = extract_features(test_with_index[["date", "index", "global_rad:W"]], 
                                                        column_id="index", 
                                                        column_sort="date", 
                                                        default_fc_parameters=EfficientFCParameters())

            
            
            impute(extracted_train_features)
            features_filtered = select_features(extracted_train_features, train_with_index["pv_measurement"], ml_task="regression")
            valid_test_cols = pd.DataFrame()
            for col in features_filtered:
                valid_test_cols[col] = extracted_test_features[col]
                
            final_train = pd.merge(train_with_index, features_filtered.reset_index(), on="index")
            final_train.drop("index", axis=1, inplace=True)
            
            final_test = pd.merge(test_with_index, valid_test_cols.reset_index(), on="index")
            final_test.drop("index", axis=1, inplace=True)
            
            new_train.append(final_train)
            new_test.append(final_test)
            
        self.training_data = new_train
        self.test_data = new_test


        train_data_a, val_data_a = self.split_data(self.training_data[0])
        train_data_b, val_data_b = self.split_data(self.training_data[1])
        train_data_c, val_data_c = self.split_data(self.training_data[2])
        
        train_data_a.drop(["date"], axis=1, inplace=True)
        val_data_a.drop(["date"], axis=1, inplace=True)
        self.test_data[0].drop(["date"], axis=1, inplace=True)

        train_data_b.drop(["date"], axis=1, inplace=True)
        val_data_b.drop(["date"], axis=1, inplace=True)
        self.test_data[1].drop(["date"], axis=1, inplace=True)


        train_data_c.drop(["date"], axis=1, inplace=True)
        val_data_c.drop(["date"], axis=1, inplace=True)
        self.test_data[2].drop(["date"], axis=1, inplace=True)
        
        
        
        
        
        train_a_y = train_data_a.pop("pv_measurement")
        val_a_y = val_data_a.pop("pv_measurement")

        train_b_y = train_data_b.pop("pv_measurement")
        val_b_y = val_data_b.pop("pv_measurement")

        train_c_y = train_data_c.pop("pv_measurement")
        val_c_y = val_data_c.pop("pv_measurement")
        

        
        self.A = Building(train_data_a, train_a_y, val_data_a, val_a_y, self.test_data[0])
        self.B = Building(train_data_b, train_b_y, val_data_b, val_b_y, self.test_data[1])
        self.C = Building(train_data_c, train_c_y, val_data_c, val_c_y, self.test_data[2])
